In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
import astropy.units as u
from astropy.units import cds
cds.enable()
import pandas as pd
import lightkurve as lk
import random

In [ ]:
# Determine size of day/night portions

A_data = 9*u.h #hours
GapA = 15*u.h #hours

A_secs = A_data.to(u.s)
GapA_secs = GapA.to(u.s)

print(A_secs, GapA_secs)

In [ ]:
# Make sine wave

time = np.arange(0, 205200, 30)

def sine(x, amp = 0.0001, period = 500, phase = 0):
    
    return amp*np.sin(x*((2*np.pi)/period)+phase)

flux = sine(time)

In [ ]:
# Make the gap sine wave

time_gap = np.arange(0, 205200, 30)

def sine(x, amp = 0.0001, period = 500, phase = 0):
    
    return amp*np.sin(x*((2*np.pi)/period)+phase)

flux_gap = sine(time)

In [ ]:
# Create the gaps in the gap sine wave

for i in range(0, len(time_gap)):
    if 32400 < time_gap[i] <= 86400:
        flux_gap[i] = np.nan
    elif 118800 < time_gap[i] <= 172800:
        flux_gap[i] = np.nan

In [ ]:
lc = lk.LightCurve(time = time/86400, flux = flux + 1).normalize(unit = 'ppm')

pg = lc.to_periodogram(freq_unit=u.microHertz)

lc_gap = lk.LightCurve(time = time_gap/86400, flux = flux_gap + 1).normalize(unit = 'ppm')

pg_gap = lc_gap.to_periodogram(freq_unit=u.microHertz)

In [ ]:
fig, ax = plt.subplots(1, 2, sharex = True, sharey = True, figsize = (8, 4))

plt.rcParams.update({'font.size': 12})

ax[0].plot(pg_gap.frequency, pg_gap.power, color = 'palevioletred')
ax[1].plot(pg.frequency, pg.power, color = 'palevioletred')

ax[0].set_xlim(1801, 2199)
ax[1].set_xlim(1801, 2199)

ax[0].set_ylim(0, 105)

ax[0].set_xlabel(r'Frequency ($\mu$Hz)')
ax[1].set_xlabel(r'Frequency ($\mu$Hz)')

ax[0].set_ylabel(r'Amplitude (mma)')

fig.tight_layout()
plt.savefig('aliaspattern.png', dpi = 250)

plt.show()